In [1]:
import pickle
import pandas as pd
import numpy as np

## 데이터 불러오기

In [2]:
# pickle 파일 불러오기
with open('../processed/export_GDP_pop_fx_wti.pkl', 'rb') as f:
    ratings_all = pickle.load(f)

ratings_all

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000


In [3]:
ratings = ratings_all[ratings_all['year']==2013]
ratings = ratings[['country', '품목코드', 'preference']]
# column명 변경
ratings.columns=['country_id', 'item_id', 'preference']

# country에 고유번호 부여하기
country_to_idx = {v:k for k,v in enumerate(ratings['country_id'].unique())}
idx_to_country = {v:k for k,v in country_to_idx.items()}

# country_id를 고유번호로 바꾸기
ratings['country_id'] = ratings['country_id'].map(country_to_idx)

# item_id int로 바꾸기
ratings['item_id'] = ratings['item_id'].astype(int)

# 중복제거
ratings = ratings.drop_duplicates(['country_id', 'item_id'], keep='first')

# index reset
ratings.reset_index(drop=True, inplace=True)
ratings


,country_id,item_id,preference
0,0,10,4.862752e-08
1,0,16,6.321578e-07
2,0,19,1.799218e-06
3,0,20,4.862752e-08
4,0,21,6.905108e-06
...,...,...,...
11315,202,90,4.719496e-05
11316,202,94,1.099994e-06
11317,202,95,1.152374e-06
11318,202,96,1.728561e-06


In [4]:
ratings.describe()

,country_id,item_id,preference
count,11320.000000,11320.000000,1.132000e+04
mean,104.469965,50.958392,2.021059e-04
std,57.837978,27.064881,3.011644e-03
min,0.000000,1.000000,0.000000e+00
25%,56.000000,28.000000,6.201527e-08
50%,103.000000,51.000000,1.746452e-06
75%,151.000000,73.000000,1.906240e-05
max,202.000000,97.000000,2.466425e-01


## UBCF_KNN 적용하기

In [5]:
# Rating 데이터를 test, train으로 나누고 train을 full matrix로 변환
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['country_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=12)
rating_matrix = x_train.pivot(values='preference', index='country_id', columns='item_id')

# Train set의 모든 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
country_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
country_similarity = pd.DataFrame(country_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# RMSE 계산을 위한 함수
def RMSE(y_true, y_pred):
    import numpy as np
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model, k=10): 
    id_pairs = zip(x_test['country_id'], x_test['item_id'])
    y_pred = np.array([model(country, item, k) for (country, item) in id_pairs])
    y_true = np.array(x_test['preference'])
    return RMSE(y_true, y_pred)

In [6]:
# Neighbor size를 고려하는 추천
def cf_knn(country_id, item_id, neighbor_size=14): # k 디폴트 값 14
    import numpy as np
    if item_id in rating_matrix:
        sim_scores = country_similarity[country_id]                   # 현재 사용자와 다른 전체 사용자 간의 similarity 가져오기
        item_ratings = rating_matrix[item_id]                         # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        none_rating_idx = item_ratings[item_ratings.isnull()].index   # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        item_ratings = item_ratings.drop(none_rating_idx)             # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        sim_scores = sim_scores.drop(none_rating_idx)                 # 현재 영화를 평가하지 않은 사용자의 similarity값 제거

        if neighbor_size == 0:    # Neighbor size가 지정되지 않은 경우 -> KNN 사용하지 않겠다는 의미
            # 현재 품목을 평가한 모든 국가의 가중 평균값 구하기
            mean_rating = np.dot(sim_scores, item_ratings) / (sim_scores.sum() + 0.0001)  # rating 존재하는 모든 국가 사용(simple CF사용)
        else:    # Neighbor size가 지정된 경우
            # if len(sim_scores) > 1 :
            neighbor_size = min(neighbor_size, len(sim_scores))          # 내가 지정한 K 숫자보다 품목 선호도가 있는 국가의 수가 적을 수 있음. 그럴 때, 지정된 neighbor size 값과 해당 품목 선호도가 존재하는 총국가 수 중 작은 걸 K로 결정(에러방지)
            sim_scores = np.array(sim_scores)                            # argsort()를 사용하기 위해서 array형식으로 바꿔주기
            item_ratings = np.array(item_ratings)
            country_idx = np.argsort(sim_scores)                         # 다른 국가를 유사도 순서대로 정렬(오름차순)
            sim_scores = sim_scores[country_idx][-neighbor_size:]        # 가장 유사도가 높은 k개의 국가를 선정
            item_ratings = item_ratings[country_idx][-neighbor_size:]    # 선정된 k개 국가의 주어진 품목(item_id)에 대한 평점을 추출한다
            # 최종 예측값 계산
            mean_rating = np.dot(sim_scores, item_ratings) / (sim_scores.sum() + 0.0001)  # 선정된 k 국가의 해당 품목에 대한 선호도를 유사도로 가중한 평점 평균 계산
    else:
        mean_rating = 0.000001746452   # train-set에 현재 품목에 대한 선호도가 없는 경우, 중간값을 기본값으로 지정
    return mean_rating

In [7]:
score(cf_knn, 14)

0.0018438090904163804

## ubcf_knn으로 추천해보기

In [8]:
#### 전체 데이터 불러오기 ####
ratings = ratings_all[ratings_all['year']==2013]
ratings = ratings[['country', '품목코드', 'preference']]
# column명 변경
ratings.columns=['country_id', 'item_id', 'preference']

# country에 고유번호 부여하기
country_to_idx = {v:k for k,v in enumerate(ratings['country_id'].unique())}
idx_to_country = {v:k for k,v in country_to_idx.items()}

# country_id를 고유번호로 바꾸기
ratings['country_id'] = ratings['country_id'].map(country_to_idx)

# item_id int로 바꾸기
ratings['item_id'] = ratings['item_id'].astype(int)

# 중복제거
ratings = ratings.drop_duplicates(['country_id', 'item_id'], keep='first')

# index reset
ratings.reset_index(drop=True, inplace=True)

# 품목 이름 가져오기
items_df = pd.read_csv('../processed/item_name-code.csv')
items_df.columns=['title', 'item_id']
items_df = items_df.set_index('item_id')

In [9]:
# Cosine similarity 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
country_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
country_similarity = pd.DataFrame(country_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [10]:
# 추천하기
def recommender(country, n_items=5, neighbor_size=16):                              # 주어진 국가와 유사도가 제일 높은 국가 k개를 써서 예측 rating이 높은 5개만 출력하기
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []                                                                # 예상 선호도를 저장할 리스트 만들기
    rated_index = rating_matrix.loc[country][rating_matrix.loc[country] > 0].index  # 현재 국가에 선호도가 존재하는 품목 확인(추천에서 제외하기 위함)
    items = rating_matrix.loc[country].drop(rated_index)                            # 이미 선호도가 있는 품목은 제거
    for item in items.index:                                                        # 각 품목에 대한 해당 국가의 예상 선호도 계산
        predictions.append(cf_knn(country, item, neighbor_size))                    # cf_knn함수를 호출해서 해당 국가의 현재 품목에 대한 예상 선호도를 구해서 predictions에 추가
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)   # 예상 선호도를 series형태로 변환해서 recommendations에 저장
    recommendations = recommendations.sort_values(ascending=False)[:n_items]        # 예상 선호도가 높은 순서로 정렬해서 주어진 아이템 수(n_items)만큼 선택
    recommended_items = items_df.loc[recommendations.index]['title']                # 추천 품목의 title만 뽑아냄
    return recommended_items

In [11]:
recommender(1, 5, 16)

item_id
87.0                           철도용이나 궤도용 외의 차량과 그 부분품ㆍ부속품
72.0                                                   철강
85.0    전기기기와 그 부분품, 녹음기ㆍ음성 재생기ㆍ텔레비전의 영상과 음성의 기록기ㆍ재생기와...
29.0                                                유기화학품
27.0           광물성 연료ㆍ광물유(鑛物油)와 이들의 증류물, 역청(瀝靑)물질, 광물성 왁스
Name: title, dtype: object

## Precision@K

In [26]:
# country를 index로 바꾼 pickle 파일을 불러오기.
with open('../processed/new_exp_3years.pkl', 'rb') as f:
    new_exp_3years = pickle.load(f)


In [27]:
# precision@k 함수로 만들기
def precision_at_k(country, n_items, year, neighbor_size=16):
    hit_df = ratings_all[ratings_all['year'].isin(year)]
    hit_df = hit_df[hit_df['country']==idx_to_country[country]]
    c_items = hit_df['품목코드'].unique().astype(int)
    rec_items = recommender(country, n_items, neighbor_size).index.tolist()
    hit_items = [i for i in c_items if i in rec_items]
    precision = len(hit_items) / n_items
    return precision

# 해당 년도 모든 나라에 대해서 precision@k 계산
def precision_at_k_all(n_items, year, neighbor_size):
    precision = []
    c_len = len(rating_matrix)
    for country in range(0, c_len):
        precision.append(precision_at_k(country, n_items, year, neighbor_size))
    return sum(precision) / c_len

In [49]:
# recall 함수로 만들기
def recall_at_k(country, n_items, year, neighbor_size=16):
    hit_df = ratings_all[ratings_all['year'].isin(year)]
    hit_df = hit_df[hit_df['country']==idx_to_country[country]]
    c_items = hit_df['품목코드'].unique().astype(int)
    rec_items = recommender(country, n_items, neighbor_size).index.tolist()
    hit_items = [i for i in c_items if i in rec_items]
    if len(c_items) == 0:
        recall = 0
    else:
        recall = len(hit_items) / len(c_items) # 전체 수출한 item 중에서 몇개를 추천했는지
    return recall

# 해당년도 모든 나라에 대해서 recall 계산
def recall_at_k_all(n_items, year, neighbor_size):
    recall = []
    c_len = len(rating_matrix)
    cnt = 0
    for country in range(0, c_len):
        recall.append(recall_at_k(country, n_items, year, neighbor_size))
    return sum(recall) / c_len

In [42]:
def f1_score_all(precision,recall):
    return 2 * (precision * recall) / (precision + recall)

## 연도별로 국가별 품목 추천해서 Precision@K ,Recall@K, F1 Score 계산하기

### 해당년도 이후 3개년 확인

#### 1. 2013년

In [50]:
year = [2014, 2015, 2016]
precision = precision_at_k_all(5, year, 16)
precision

0.8088669950738917

In [51]:
year = [2014, 2015, 2016]
recall = recall_at_k_all(5, year, 16)
recall

0.07074523155186845

In [52]:
f1_score_all(precision,recall)

0.13011070362375216

#### 2. 2014년

In [53]:
year = [2015, 2016, 2017]
precision = precision_at_k_all(5, year, 16)
precision

0.8118226600985218

In [54]:
year = [2015, 2016, 2017]
recall = recall_at_k_all(5, year, 16)
recall

0.07071298845358308

In [55]:
f1_score_all(precision,recall)

0.1300942494143671

#### 3. 2015년

In [56]:
year = [2016, 2017, 2018]
precision = precision_at_k_all(5, year, 16)
precision

0.8177339901477831

In [57]:
year = [2016, 2017, 2018]
recall = recall_at_k_all(5, year, 16)
recall

0.06950503059871474

In [58]:
f1_score_all(precision,recall)

0.12812021265478166

#### 4. 2016년

In [59]:
year = [2017, 2018, 2019]
precision = precision_at_k_all(5, year, 16)
precision

0.8344827586206895

In [60]:
year = [2017, 2018, 2019]
recall = recall_at_k_all(5, year, 16)
recall

0.07202170912265601

In [61]:
f1_score_all(precision,recall)

0.13259917992211584

#### 5. 2017년

In [62]:
year = [2018, 2019, 2020]
precision =precision_at_k_all(5, year, 16)
precision

0.8413793103448273

In [63]:
year = [2018, 2019, 2020]
recall = recall_at_k_all(5, year, 16)
recall

0.07304760652030115

In [64]:
f1_score_all(precision,recall)

0.13442461866081817

#### 6. 2018년

In [65]:
year = [2019, 2020, 2021]
precision = precision_at_k_all(5, year, 16)
precision

0.8354679802955663

In [66]:
year = [2019, 2020, 2021]
recall = recall_at_k_all(5, year, 16)
recall

0.07199723201864201

In [67]:
f1_score_all(precision,recall)

0.1325701111298552